In [0]:
import tensorflow as tf
import numpy as np
import sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random
import matplotlib.cm as cm
import math

# Problem 1

Have used HE initialisation and AdamOptimizer
The Layer structure is,
1. Conv 1x4 and 32 filters and stride 1
2. Max pool 1x2  and stride 1x2
3. Conv 1x2, 64 filters and stride 1
4. Maxpool 1x2 and stride 1x2
5. Conv 1x2 , 128 filters and stride 1
6. Fully connected layers [128x129,800, 513]


In [0]:
###load files
# !pip install librosa # in colab, you'll need to install this
import librosa
import io


"""
Change directory path here and leave it blank if its in the same directory as the 
Notebook
"""
# dirpath  = "./drive/DeepLearning Assign1/"
dirpath  = ""


### Loading the files

##Y path  = 
sn_train_y, sr=librosa.load(dirpath+'train_clean_male.wav', sr=None)
S_Train=librosa.stft(sn_train_y, n_fft=1024, hop_length=512)

##X
sn_train_x, sr=librosa.load(dirpath+'train_dirty_male.wav', sr=None)
X_Train=librosa.stft(sn_train_x, n_fft=1024, hop_length=512)

##XTest1
sn_test1, sr=librosa.load(dirpath+'test_x_01.wav', sr=None)
X1_test =librosa.stft(sn_test1, n_fft=1024, hop_length=512)


##XTest2
sn_test2, sr=librosa.load(dirpath+'test_x_02.wav', sr=None)
X2_test =librosa.stft(sn_test2, n_fft=1024, hop_length=512)

X1_test = np.transpose(X1_test)
X1_Test_Mag = np.abs(X1_test)

X2_test = np.transpose(X2_test)
X2_Test_Mag = np.abs(X2_test)

S_Mag = np.transpose(np.abs(S_Train))
X_Mag = np.transpose(np.abs(X_Train))

# print(X1_test.shape)

In [0]:


##network params
learning_rate = tf.constant(0.0003)
numEpochs = 400
batchSize = 100
displayStep = 100
                  #list of nodes  excpet x and y

(numExamples,xDim) = X_Mag.shape
(numExamples,yDim) = S_Mag.shape


X = tf.placeholder(tf.float32,[None,xDim],name = "X")
Y  =  tf.placeholder(tf.float32,[None,yDim], name = "Y")



## collecting weights in a dictionary

#filterHeight, filterWidth, inputDepth, numberOfFilters
AllfilterDims = [(1, 4, 1, 32),(1, 2, 32, 64),(1, 2, 64, 128)]

## (to apply or not, height,width) ## reduces by the stride in height and width
MaxPoolLayers = [(False,1,2),(True,1,2),(True,1,2)] ##to be appplied after the first filter



### fully connected definitions
AllFullyConnectedDims = [128*129,800] ##does not include ydim


def getFinalFullyConDims(inputDims,AllfilterDims,MaxPoolLayer):
   
  
  pass


def filterWeigths(filterHeight, filterWidth, inputDepth, numberOfFilters):
  ## easy func for intiailiisation of weights
  initialiser = tf.contrib.layers.variance_scaling_initializer()
  
  filterWeightTensor = tf.Variable(initialiser([filterHeight,filterWidth,inputDepth,numberOfFilters],dtype=tf.float32))
  return filterWeightTensor

def filterBisases(numberOfFilters):
  ## easy functio for initiliasation of biases
  initialiser = tf.contrib.layers.variance_scaling_initializer()
  filterBiasTensor = tf.Variable(initialiser([numberOfFilters],dtype=tf.float32)) 
  return filterBiasTensor


def defineWeightsAndBias(AllfilterDims,AllFullyConnectedDims,yDim):
  
  ## he initialiisation
  initialiser = tf.contrib.layers.variance_scaling_initializer()
  
  
  
  ## filtre weights and biasses
  FilterWeights = [filterWeigths(filterHeight, filterWidth, inputDepth, numberOfFilters) \
             for (filterHeight, filterWidth, inputDepth, numberOfFilters) in AllfilterDims]
  
  
  Filterbiases = [filterBisases(numberOfFilters) for (filterHeight, filterWidth, inputDepth, numberOfFilters) in AllfilterDims]

  
  
  ## fully connected weights and biases
  FullyConWeights = [tf.Variable(initialiser([j,k],dtype=tf.float32)) \
                      for (j,k) in zip(AllFullyConnectedDims,AllFullyConnectedDims[1:]+[yDim])]
  
  
  FullyConBiases = [tf.Variable(initialiser([j],dtype=tf.float32))
                      for j in AllFullyConnectedDims[1:]+[yDim]]
  
  
  
  return FilterWeights, Filterbiases,FullyConWeights,FullyConBiases ##contains y incluede weights



def convolve2d(inputToLayer, singleFilterWeight, singleFilterBias, strides=1):
    #conv2 easy func
    convOutput = tf.nn.conv2d(inputToLayer, singleFilterWeight, strides=[1, strides, strides, 1], padding='SAME')
    convOutput = tf.nn.bias_add(convOutput, singleFilterBias)
    return tf.nn.relu(convOutput)


def maxpool2d(inputToLayer, filterSizeHeight,filterSizeWidth):
#     maxpool2d easy func
    return tf.nn.max_pool(inputToLayer, ksize=[1, filterSizeHeight, filterSizeWidth, 1], strides=[1, filterSizeHeight, filterSizeWidth, 1],
                          padding='SAME')





In [0]:
## accuracy metirc 
def RSquared(yAct, yPred):

    residual = tf.reduce_sum(tf.square(tf.subtract(yAct, yPred)))
    total = tf.reduce_sum(tf.square(tf.subtract(yAct, tf.reduce_mean(yAct))))
    r2 = tf.subtract(1.0, tf.math.divide(residual, total))
    return r2
  
  
## convert out put to time domian
def getOutputSound(xRawInp,xMag, YPred):
    S_Hat = np.multiply(xRawInp/xMag,YPred)
    S_Hat = librosa.istft(S_Hat.T, hop_length=512)

        
    return S_Hat
  
## signal ton noise ration
def getSNR(yRawInpTimeDomain,yPredReadyToWrite):
    minLength = min(len(yRawInpTimeDomain),len(yPredReadyToWrite))
    clippedY_True = yRawInpTimeDomain[:minLength]
    clippedY_Pred = yPredReadyToWrite[:minLength]
    signal  = np.dot(clippedY_True,clippedY_True.T)
    subtract = clippedY_True - clippedY_Pred
    noise = np.dot(subtract,subtract.T)
    
    return 10*math.log10(signal/noise)

In [0]:
def getNetworkOutput(xInp,FilterWeights, Filterbiases,FullyConWeights,FullyConBiases,MaxPoolLayer):
    ##-1 to infer batch size, Height, Width, Depth
      
    xInp = tf.reshape(xInp, shape=[-1, 1, 513, 1])
    totalLayers = len(FilterWeights)+len(FullyConWeights)+ 1 ##for y 
    
    ## declarer empty list to save all outputs
    
    filterOutputList = []
    FullConOutputList = [0 for i in  range(len(FullyConWeights))]

    ## drive through filters
       
    
    for layerPosition,(filWeight,filBias,(boolMaxpool,maxPoolHeight,maxPoolwidth)) in enumerate(zip(FilterWeights,Filterbiases,MaxPoolLayers)):
      
#       tf.print(layerPosition)
      if layerPosition == 0:
        
        output = convolve2d(xInp, filWeight, filBias)
      
      else:
        
        output = convolve2d(output, filWeight, filBias)
        
        filterOutputList.append(output) ##collecting filter outputs
      
      if boolMaxpool == True:
        
        output = maxpool2d(output,maxPoolHeight,maxPoolwidth)
        
        
      filterOutputList.append(output) ##collecting max pool outputs
      

      maxPoolReshaped = tf.reshape(output, [-1, FullyConWeights[0].get_shape().as_list()[0]])
      



  ###drive through fully connected
    
    
    lastLayerPosition = len(FullyConWeights)-1
    for layerPosition,(weight,bias) in enumerate(zip(FullyConWeights,FullyConBiases)):

      if layerPosition == 0:
      
        FullConOutputList[layerPosition] = tf.nn.relu(tf.add(tf.matmul(maxPoolReshaped, weight), bias))
         


      elif layerPosition <= lastLayerPosition-1:
      
        FullConOutputList[layerPosition] = tf.nn.relu(tf.add(tf.matmul(FullConOutputList[layerPosition-1], weight), bias))
        
      
      else:
        
        FullConOutputList[layerPosition] = tf.add(tf.matmul(FullConOutputList[layerPosition-1], weight), bias)
        FullConOutputList[layerPosition] = tf.nn.relu(FullConOutputList[layerPosition],name='activation_relu')
       
    
    
    
    return FullConOutputList[-1],filterOutputList,FullConOutputList




In [0]:
## required training
FilterWeights, Filterbiases,FullyConWeights,FullyConBiases = defineWeightsAndBias(AllfilterDims,AllFullyConnectedDims,yDim)

FinalOutput ,filterOutputList,FullConOutputList = getNetworkOutput(X,FilterWeights, Filterbiases,FullyConWeights,FullyConBiases,MaxPoolLayers)
print(filterOutputList)
# layer_Output, AllLayerOutputs = getNetworkOutput(X,nodesInEachHiddenLayer,yDim)
lossCalcu  = tf.losses.mean_squared_error(labels = Y ,predictions = FinalOutput)

AdamOptimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train = AdamOptimizer.minimize(lossCalcu)
accuracy = RSquared(FinalOutput,Y)

initialise = tf.global_variables_initializer()

In [7]:
### training and running the sessions

with tf.Session() as currSess:


    x_train = X_Mag
    

    y_train = S_Mag
    
    
    
    
    ##training 
    currSess.run(initialise)
    
    
    for epoch in range(500):
      for index, offset in enumerate(range(0, len(x_train), batchSize)):
        
        xBatch, yBatch = x_train[offset: offset + batchSize], y_train[offset: offset + batchSize]

          
        currSess.run(train,feed_dict ={X:xBatch,Y:yBatch})
        
      loss = currSess.run([lossCalcu], feed_dict={X: x_train, Y: y_train})  
      accu = currSess.run([accuracy], feed_dict={X: x_train, Y: y_train})  
#       accu = 0
      
      print("Step " + str(epoch) + ", Loss= " + str(loss) + " Accuracy = "+ str(accu))
      
    ###predictions for test sets
    x_test_Mag = X1_Test_Mag
    testPreds1 = currSess.run([FinalOutput], feed_dict={X: x_test_Mag})  
    s_Hat1 = getOutputSound(X1_test,x_test_Mag,testPreds1)
    
    
    x_test_Mag = X2_Test_Mag
    testPreds2 = currSess.run([FinalOutput], feed_dict={X: x_test_Mag})  
    s_Hat2 = getOutputSound(X2_test,x_test_Mag,testPreds2)
    
    
    trainPreds = currSess.run([FinalOutput], feed_dict={X: X_Mag})  
    s_Hat_Train = getOutputSound(X_Train.T,X_Mag,trainPreds)
    print(getSNR(sn_train_y,s_Hat_Train))
    
    
    
    


Step 0, Loss= [0.06008529] Accuracy = [-0.6744983]
Step 1, Loss= [0.048820034] Accuracy = [0.06453574]
Step 2, Loss= [0.042512156] Accuracy = [0.15650511]
Step 3, Loss= [0.04020303] Accuracy = [0.32386017]
Step 4, Loss= [0.040163714] Accuracy = [0.3544104]
Step 5, Loss= [0.039778277] Accuracy = [0.35797286]
Step 6, Loss= [0.03896656] Accuracy = [0.3656323]
Step 7, Loss= [0.038474504] Accuracy = [0.36451983]
Step 8, Loss= [0.037381496] Accuracy = [0.38992912]
Step 9, Loss= [0.038785856] Accuracy = [0.2956938]
Step 10, Loss= [0.036738038] Accuracy = [0.36904454]
Step 11, Loss= [0.035091877] Accuracy = [0.4626773]
Step 12, Loss= [0.034844752] Accuracy = [0.4568097]
Step 13, Loss= [0.03393797] Accuracy = [0.46846402]
Step 14, Loss= [0.034086704] Accuracy = [0.45795226]
Step 15, Loss= [0.03304321] Accuracy = [0.47495955]
Step 16, Loss= [0.033770934] Accuracy = [0.43434548]
Step 17, Loss= [0.03277551] Accuracy = [0.5269144]
Step 18, Loss= [0.03344677] Accuracy = [0.5271545]
Step 19, Loss= [0

In [0]:
### write to required files

librosa.output.write_wav('test_s_01_recons.wav', s_Hat1, sr = 16000)
librosa.output.write_wav('test_s_02_recons.wav', s_Hat2, sr = 16000)
librosa.output.write_wav('train_cleaned.wav', s_Hat_Train, sr = 16000)
